In [2]:
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

In [4]:
from rpy2.robjects.packages import importr

utils = importr("utils")
utils.chooseCRANmirror(ind=1)
utils.install_packages('lme4')

%load_ext rpy2.ipython

R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/lme4_1.1-27.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 3311365 bytes (3.2 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to con

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [5]:
# Load data
conversations = pd.read_csv("results/intelligibility/conversations.csv")
conversations_melted = pd.read_csv("results/intelligibility/conversations_melted.csv")

# convert True/False to 0/1:
conversations.replace({False: 0, True: 1}, inplace=True)
conversations_melted.replace({False: 0, True: 1}, inplace=True)

conversations.head()


,utterance_id,speaker_code,transcript_raw,tokens,pos,start_time,end_time,age,corpus,transcript_file,...,follow_up_transcript_raw,follow_up_start_time,follow_up_is_speech_related,follow_up_is_intelligible,follow_up_speech_act,response_latency,response_latency_follow_up,has_response,response_is_clarification_request,pos_feedback
0,28,CHI,let's get some glue too .,"[""let's"", 'get', 'some', 'glue', 'too', '.']","['v', 'pro:obj', 'v', 'qn', 'n', 'adv']",92795.0,96687.0,30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,...,let's get some glue don't touch that .,97154.0,1,1,PF,0.0,467.0,1,0,1
1,50,CHI,xxx I get some glue get some glue too .,"['i', 'get', 'some', 'glue', 'too', '.']","['pro:sub', 'v', 'qn', 'n', 'adv']",141382.0,148262.0,30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,...,xxx broken .,150114.0,1,1,AA,0.0,1852.0,1,0,1
2,52,CHI,xxx broken .,"['broken', '.']",['part'],150114.0,151434.0,30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,...,xxx table's leg .,155422.0,1,1,ST,0.0,3988.0,1,0,1
3,58,CHI,huh right there see his legs see table's legs .,"['huh', 'right', 'there', 'see', 'his', 'legs'...","['co', 'adv', 'adv', 'v', 'det:poss', 'n', 'v'...",163402.0,169064.0,30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,...,right .,172202.0,1,1,AD,0.0,3138.0,1,0,1
4,78,CHI,hi Jenny how you fine hi how you doing .,"['hi', 'jenny', 'how', 'you', 'fine', 'hi', 'h...","['co', 'n:prop', 'pro:rel', 'pro:per', 'adv', ...",212706.0,243831.0,30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,...,right .,249125.0,1,1,AA,0.0,5294.0,1,0,1


In [17]:
# Some children have very few data points:
counts = conversations.groupby("corpus").size()
counts

corpus
Bloom                   1042
Braunwald               1948
Brent                   2076
Forrester                205
Gleason                 6628
MPI-EVA-Manchester    139924
MacWhinney              9672
NewmanRatner             140
Peters                  5363
Providence             46594
Rollins                  113
Sachs                   2596
Snow                    6105
Soderstrom              1404
Thomas                111165
Tommerdahl              5287
Weist                   8395
dtype: int64

In [8]:
# normalize age
min_age, max_age = conversations["age"].min(), conversations["age"].max()
conversations["age"] = (conversations["age"] - min_age) / (max_age - min_age) * (1 - 0)

## Quality of communicative feedback/ Caregiver contingency


### Timing:

In [27]:
%%R -i conversations
library(lme4)

m_caregiver_contingency<-glmer('has_response ~ is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: has_response ~ is_intelligible * age + (1 | child_name)
   Data: conversations

     AIC      BIC   logLik deviance df.resid 
180541.4 180592.1 -90265.7 180531.4   187471 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.6210  0.1439  0.4934  0.5347  0.9662 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 1.356    1.165   
Number of obs: 187476, groups:  child_name, 31

Fixed effects:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)          1.78962    0.15349  11.660   <2e-16 ***
is_intelligible      0.99341    0.04343  22.874   <2e-16 ***
age                  1.15514    0.08013  14.416   <2e-16 ***
is_intelligible:age -0.13875    0.08246  -1.683   0.0924 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) is_ntl a

### Clarification requests

In [22]:
%%R -i conversations
library(lme4)

# TODO: filter out no response cases?
m_caregiver_contingency<-glmer('response_is_clarification_request ~ is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: response_is_clarification_request ~ is_intelligible * age + (1 |  
    child_name)
   Data: conversations

     AIC      BIC   logLik deviance df.resid 
 15664.5  15715.2  -7827.2  15654.5   187798 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-0.358 -0.073 -0.070 -0.068 37.636 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 1.228    1.108   
Number of obs: 187803, groups:  child_name, 39

Fixed effects:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)          -2.9402     0.2498 -11.770  < 2e-16 ***
is_intelligible      -2.4388     0.1314 -18.567  < 2e-16 ***
age                  -1.4524     0.2158  -6.730 1.70e-11 ***
is_intelligible:age   1.0451     0.2356   4.436 9.14e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
          

### Combined (Positive Feedback = No pause, no clarification request)

In [11]:
%%R -i conversations
library(lme4)

m_caregiver_contingency<-glmer('pos_feedback ~ is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: pos_feedback ~ is_intelligible * age + (1 | child_name)
   Data: conversations

     AIC      BIC   logLik deviance df.resid 
184200.0 184250.8 -92095.0 184190.0   187829 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-7.2747  0.1601  0.4997  0.5418  1.0355 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 0.8515   0.9228  
Number of obs: 187834, groups:  child_name, 40

Fixed effects:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)          1.33834    0.14175   9.441   <2e-16 ***
is_intelligible      1.12552    0.04114  27.360   <2e-16 ***
age                  1.21939    0.07799  15.636   <2e-16 ***
is_intelligible:age -0.19354    0.07821  -2.475   0.0133 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) is_ntl a

## Effect of communicative feedback
### Positive Feedback: Timing

In [12]:
%%R -i conversations
library(lme4)

# conversations_child_intelligible = subset(conversations, is_intelligible==1)
# TODO: only intelligible ones?

m_child_contingency<-glmer('follow_up_is_intelligible ~ has_response * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_child_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: follow_up_is_intelligible ~ has_response * age + (1 | child_name)
   Data: conversations

     AIC      BIC   logLik deviance df.resid 
105692.5 105743.2 -52841.2 105682.5   187829 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.6859  0.2320  0.2554  0.3281  3.7015 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 2.91     1.706   
Number of obs: 187834, groups:  child_name, 40

Fixed effects:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)       1.64205    0.15479  10.608  < 2e-16 ***
has_response      1.13774    0.04273  26.628  < 2e-16 ***
age               1.74680    0.07592  23.008  < 2e-16 ***
has_response:age -0.53273    0.08113  -6.567 5.15e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) hs_rsp age   

### Negative Feedback: Clarification requests

In [12]:
%%R -i conversations_melted
library(lme4)

conversations_melted_cr = subset(conversations_melted, response_is_clarification_request == 1)


m_child_contingency<-glmer('is_intelligible ~ is_follow_up + age + (1 | child_name) + (1 | is_follow_up)', data=conversations_melted_cr, family=binomial)
print(summary(m_child_contingency))

R[write to console]: boundary (singular) fit: see ?isSingular



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: is_intelligible ~ is_follow_up + age + (1 | child_name) + (1 |  
    is_follow_up)
   Data: conversations_melted_cr

     AIC      BIC   logLik deviance df.resid 
  4056.3   4087.4  -2023.1   4046.3     3735 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.8983 -0.8930  0.4035  0.6665  2.5446 

Random effects:
 Groups       Name        Variance Std.Dev.
 child_name   (Intercept) 1.059    1.029   
 is_follow_up (Intercept) 0.000    0.000   
Number of obs: 3740, groups:  child_name, 47; is_follow_up, 2

Fixed effects:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -2.167157   0.287775  -7.531 5.05e-14 ***
is_follow_up  0.836519   0.078459  10.662  < 2e-16 ***
age           0.086483   0.006925  12.489  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            

In [50]:
%%R -i conversations_melted
library(lme4)

m_child_contingency<-glmer('is_intelligible ~ response_is_clarification_request * is_follow_up + (1 | age) + (1 | child_name) + (1 | is_follow_up)', data=conversations_melted, family=binomial)
print(summary(m_child_contingency))

R[write to console]: boundary (singular) fit: see ?isSingular



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: is_intelligible ~ response_is_clarification_request * is_follow_up +  
    (1 | age) + (1 | child_name) + (1 | is_follow_up)
   Data: conversations_melted

      AIC       BIC    logLik  deviance  df.resid 
 382871.7  382951.9 -191428.9  382857.7    696897 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.2339  0.2274  0.2477  0.3078 12.4579 

Random effects:
 Groups       Name        Variance Std.Dev.
 child_name   (Intercept) 2.786    1.669   
 age          (Intercept) 1.418    1.191   
 is_follow_up (Intercept) 0.000    0.000   
Number of obs: 696904, groups:  child_name, 51; age, 8; is_follow_up, 2

Fixed effects:
                                                Estimate Std. Error z value
(Intercept)                                     1.393185   0.070020  19.897
response_is_clarification_request              -1.908779   0.037073 -51.487